<a href="https://colab.research.google.com/github/MatienkoAndrew/notebooks/blob/main/%22SecondStepNLP_Fasttext_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Классификация текстов

В этом ноутбуке мы разберем задачу классификации текстов на примере соревнования по выявлению токсичных твиттов: https://www.kaggle.com/competitions/toxic-comments-classification-2

Наша задача - построить классификатор, который по тексту твитта определяет, токсичный он или нет.

__План:__
1. Разбираем/освежаем в памяти простые бейзлайны: мешок слов, TF-IDF. 
2. Обучаем классификатор на основе w2v эмбеддингов
3. Знакомимся с моделью fastText

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

In [6]:
SEED = 42
np.random.seed(SEED)

Обучающая выборка и тестовые данные для предикта - их можно скачать с Kaggle.

In [2]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [10]:
np.random.seed(SEED)
train.sample(3)

,comment,toxic
5100,"Учите матчасть, селекционер.\n",0.0
6500,"А бандиты там как, понты любят?\n",1.0
9750,"у электрички габарит Т, у нас 1-ВМ ,и в центра...",0.0


In [12]:
np.random.seed(SEED)
test.sample()

,comment_id,comment
3027,3027,Быдло сестра называет девушку парня тупой шлюх...


In [13]:
#y_test = pd.read_csv('test_labels.csv')

В моем распоряжении также есть файл с тестовыми метками классов - если Вы запускаете ноутбук и хотите обучить модель, разбейте train выборку на train и test, раскомментировав строчки кода в ячейке ниже:

In [22]:
train, test = train_test_split(train, test_size=0.3)
y_test = test[['comment', 'toxic']]

test.drop(columns=['toxic'], inplace=True)

# train, test = train_test_split(x_train, test_size=0.3)
# y_test = test[['comment_id', 'toxic']]

# test.drop(columns=['toxic'], inplace=True)

In [23]:
y_test.sample()

,comment,toxic
228,"в новокузнецком Пусть радуется, что не заточко...",1.0


In [24]:
y_test = y_test['toxic'].values
y_test

array([1., 0., 1., ..., 1., 0., 0.])

### Мы начнем с простых бейзлайнов

Это всегда хорошая практика - сперва попробовать что-то предельно простое (: В нашем случае это будет логистическая регрессия + мешок слов (Bag of Words, BoW).

In [25]:
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer

In [26]:
vec = CountVectorizer(ngram_range=(1, 1), token_pattern='\w{3,}') # строим BoW для слов

# vec = CountVectorizer(ngram_range=(1, 1), token_pattern='\w{3,}') # строим BoW для слов

In [ ]:
help(CountVectorizer)

In [31]:
%%time

bow = vec.fit_transform(train['comment'])
# bow = vec.fit_transform(train['comment'])

CPU times: user 248 ms, sys: 7.49 ms, total: 255 ms
Wall time: 262 ms


In [55]:
train.shape

(5296, 2)

In [32]:
bow

<5296x35100 sparse matrix of type '<class 'numpy.int64'>'
	with 104031 stored elements in Compressed Sparse Row format>

In [39]:
print(train.comment[8334])

а что делать если сортиров в стране как не было так и нет .



In [40]:
list(vec.vocabulary_.items())[:10]

[('однако', 17797),
 ('кронштадтом', 12644),
 ('также', 30187),
 ('путаюсь', 24745),
 ('поверите', 20713),
 ('принципе', 23551),
 ('армии', 1689),
 ('зарплата', 9378),
 ('терпимая', 30560),
 ('вот', 4374)]

In [42]:
sorted(list(vec.vocabulary_.items()), key=lambda x: x[0])[:10]

[('000', 0),
 ('030050', 1),
 ('0611', 2),
 ('068', 3),
 ('0849', 4),
 ('100', 5),
 ('1000', 6),
 ('10000', 7),
 ('100000км', 8),
 ('1000рублей', 9)]

In [43]:
list(vec.vocabulary_.keys())[:10]

['однако',
 'кронштадтом',
 'также',
 'путаюсь',
 'поверите',
 'принципе',
 'армии',
 'зарплата',
 'терпимая',
 'вот']

In [44]:
len(vec.vocabulary_.items())

35100

In [45]:
y_train = train['toxic'].astype(int).values
y_train

array([0, 0, 0, ..., 0, 1, 0])

In [48]:
clf = LogisticRegression(random_state=42, max_iter=500) #
clf.fit(bow, y_train)

LogisticRegression(max_iter=500, random_state=42)

In [ ]:
help(clf.fit)

In [50]:
len(clf.coef_[0])

35100

In [51]:
bow_test = vec.transform(test['comment'])
bow_test

<2270x35100 sparse matrix of type '<class 'numpy.int64'>'
	with 33861 stored elements in Compressed Sparse Row format>

In [52]:
pred = clf.predict(bow_test)
pred[:10]

array([0, 0, 1, 0, 0, 1, 0, 0, 0, 1])

In [53]:
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.93      0.84      0.88      1736
           1       0.60      0.79      0.68       534

    accuracy                           0.83      2270
   macro avg       0.76      0.81      0.78      2270
weighted avg       0.85      0.83      0.83      2270



### Попробуем добавить препроцессинг текста

Препроцессинг, как правило, включает удаление небуквенных символов, топ-слов и нормализацию (стемминг - приведение к основе слова - или лемматизацию - приведение слов к начальной форме).

Кроме того, заменим мешок слов на TF-IDF матрицу. В качестве модели оставим логистическую регрессию.

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer

In [57]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 52.9 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=0fcf700d5a37f1fc479ba56b42b02b4747f0773735074e4b79f3f78eb7171ade
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [58]:
import nltk
nltk.download('stopwords')

#import nltk
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Функция для удаления небуквенных символов из текста:

In [59]:
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

from functools import lru_cache
from tqdm.notebook import tqdm

m = MorphAnalyzer()
regex = re.compile("[а-яa-zёЁ]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

# import re
# from pymorphy2 import MorphAnalyzer
# from nltk.corpus import stopwords

# from functools import lru_cache
# from tqdm.notebook import tqdm

# m = MorphAnalyzer()
# regex = re.compile("[а-яa-zёЁ]+")

# def words_only(text, regex=regex):
#     try:
#         return regex.findall(text.lower())
#     except:
#         return []

In [60]:
train.comment[1]

'И именно эти неработающие весы показывают, что работающих нет?..\n'

In [61]:
words_only(train.comment[1])

['и',
 'именно',
 'эти',
 'неработающие',
 'весы',
 'показывают',
 'что',
 'работающих',
 'нет']

Функции для препроцессинга текста: 

1. Удаление небуквенных символов
2. Лемматизация 
3. Удаление коротких (менее 3 символов) и стоп-слов

In [64]:
@lru_cache(maxsize=128)
def lemmatize_word(token: str, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text: list):
    return [lemmatize_word(w) for w in text]

mystopwords = stopwords.words('russian')
def remove_stopwords(lemmas: list, stopwords=mystopwords):
    return [w for w in lemmas if w not in stopwords and len(w) > 3]

def clean_text(text: str):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    return ' '.join(remove_stopwords(lemmas))

# @lru_cache(maxsize=128)
# def lemmatize_word(token, pymorphy=m):
#     return pymorphy.parse(token)[0].normal_form

# def lemmatize_text(text):
#     return [lemmatize_word(w) for w in text]


# mystopwords = stopwords.words('russian') 
# def remove_stopwords(lemmas, stopwords = mystopwords):
#     return [w for w in lemmas if not w in stopwords and len(w) > 3]

# def clean_text(text):
#     tokens = words_only(text)
#     lemmas = lemmatize_text(tokens)
    
#     return ' '.join(remove_stopwords(lemmas))

In [65]:
%time lemmatize_word('неработающие')

CPU times: user 852 µs, sys: 0 ns, total: 852 µs
Wall time: 1.21 ms


'неработающий'

In [66]:
train.comment[1]

'И именно эти неработающие весы показывают, что работающих нет?..\n'

In [67]:
clean_text(train.comment[1])

'именно неработающий весы показывать работать'

Проводим препроцессинг для train и test выборок:

In [70]:
lemmas = list(tqdm(map(clean_text, train['comment']), total=len(train)))

train['lemmas'] = lemmas
train.sample(5)

# lemmas = list(tqdm(map(clean_text, train['comment']), total=len(train)))
    
# train['lemmas'] = lemmas
# train.sample(5)

  0%|          | 0/5296 [00:00<?, ?it/s]

,comment,toxic,lemmas
8857,Зачем тут эта хуета про кастрюли?\n,1.0,хует кастрюля
9961,"А если аллергия, ещё и звуковые спецэффекты бу...",0.0,аллергия звуковой спецэффект
1318,"фишка в том, что социализм это не уровень выше...",0.0,фишка социализм уровень выше капитализм ветка ...
2546,сколько будет стоить отключить функцию автомат...,0.0,сколько стоить отключить функция автоматически...
8837,"Немного не так, установка агрегатируется под о...",0.0,немного установка агрегатироваться определённы...


In [71]:
lemmas_test = list(tqdm(map(clean_text, test['comment']), total=len(test)))
test['lemmas'] = lemmas_test

# lemmas_test = list(tqdm(map(clean_text, test['comment']), total=len(test)))
    
# test['lemmas'] = lemmas_test

  0%|          | 0/2270 [00:00<?, ?it/s]

Считаем TF-IDF матрицу и обучаем модель:

In [73]:
vec = TfidfVectorizer(ngram_range=(1, 2)) # строим BoW для слов
tfidf = vec.fit_transform(train['lemmas'])

clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(tfidf, y_train)

pred = clf.predict(vec.transform(test['lemmas']))
accuracy_score(pred, y_test)

# vec = TfidfVectorizer(ngram_range=(1, 2)) # строим BoW для слов
# tfidf = vec.fit_transform(train['lemmas'])

# clf = LogisticRegression(random_state=42, max_iter=500)
# clf.fit(tfidf, y_train)

# pred = clf.predict(vec.transform(test['lemmas']))
# accuracy_score(pred, y_test)

0.7920704845814978

In [74]:
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.98      0.78      0.87      1981
           1       0.37      0.90      0.52       289

    accuracy                           0.79      2270
   macro avg       0.68      0.84      0.70      2270
weighted avg       0.90      0.79      0.82      2270



## Word2Vec

Попробуем использовать эмбеддинги слов - для этого сперва обучим модель Word2Vec c помощью библиотеки gensim.

In [75]:
from gensim.models import word2vec

In [76]:
train.sample()

,comment,toxic,lemmas
10230,За рогатую крысу-крысу\n,1.0,рогатый крыса крыса


In [77]:
#help( word2vec.Word2Vec)

In [79]:
tokenized_tweets = [tweet.split() for tweet in train['lemmas'].values]

%time w2v = word2vec.Word2Vec(tokenized_tweets, workers=4, size=200, min_count=10, window=3, sample=1e-3)

# tokenized_tweets = [tweet.split() for tweet in train['lemmas'].values]

# %time w2v = word2vec.Word2Vec(tokenized_tweets, workers=4, vector_size=200, min_count=10, window=3, sample=1e-3)

CPU times: user 1.31 s, sys: 21.4 ms, total: 1.33 s
Wall time: 984 ms


In [80]:
w2v.wv.most_similar(positive=['плюс'], topn=10)
# w2v.wv.most_similar(positive=['плюс'], topn=10)

[('мочь', 0.9999347925186157),
 ('дать', 0.9999326467514038),
 ('именно', 0.9999306201934814),
 ('весь', 0.9999306201934814),
 ('страна', 0.9999298453330994),
 ('сделать', 0.9999297857284546),
 ('ребёнок', 0.9999287128448486),
 ('хороший', 0.9999285936355591),
 ('время', 0.9999281167984009),
 ('который', 0.9999278783798218)]

Теперь у нас есть эмбеддинги для слов. Но как получить эмбеддинги для твитов?

Можно просто усреднить эмбеддинги слов, входящих в твит.

In [82]:
def get_tweet_embedding(lemmas, model=w2v.wv, embedding_size=200):
    res = np.zeros(embedding_size)
    cnt = 0
    for word in lemmas.split():
        if word in model:
            res += np.array(model[word])
            cnt += 1
    if cnt:
        res /= cnt
    return res

# def get_tweet_embedding(lemmas, model=w2v.wv, embedding_size=200):
    
#     res = np.zeros(embedding_size)
#     cnt = 0
#     for word in lemmas.split():
#         if word in model:
#             res += np.array(model[word])
#             cnt += 1
#     if cnt:
#         res = res / cnt
#     return res

In [90]:
get_tweet_embedding('мочь дать')[:10]

array([ 0.28342253, -0.72313756, -0.41315034, -0.08118716,  0.0515172 ,
       -0.49743643,  0.04106423,  0.25147135,  0.08538125,  0.25044674])

Для каждого твита из обучающей и тестовой выборки вычислим такой эмбеддинг:

In [91]:
train['w2v_embedding'] = train['lemmas'].map(get_tweet_embedding)
test['w2v_embedding'] = test['lemmas'].map(get_tweet_embedding)
# train['w2v_embedding'] = train['lemmas'].map(get_tweet_embedding)
# test['w2v_embedding'] = test['lemmas'].map(get_tweet_embedding)

In [92]:
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(list(train['w2v_embedding'].values), y_train)

pred = clf.predict(list(test['w2v_embedding'].values))
accuracy_score(pred, y_test)

0.6903083700440529

In [93]:
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

         0.0       1.00      0.69      0.82      2270
         1.0       0.00      0.00      0.00         0

    accuracy                           0.69      2270
   macro avg       0.50      0.35      0.41      2270
weighted avg       1.00      0.69      0.82      2270



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## FastText

FastText - это модификация модели word2vec.

FastText использует не только векторы слов, но и векторы n-грам. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). Благодаря этому мы можем также получать вектора для слов, отсутствуюших в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

* [Статья](https://aclweb.org/anthology/Q17-1010)
* [Сайт](https://fasttext.cc/)
* [Руководство](https://fasttext.cc/docs/en/support.html)
* [Репозиторий](https://github.com/facebookresearch/fasttext)

Есть библиотека `fasttext` для питона (с готовыми моделями можно работать и через `gensim`).

На сайте проекта можно найти предобученные модели для 157 языков (в том числе русского): https://fasttext.cc/docs/en/crawl-vectors.html

Для начала, попробуем взять предобученную модель fastText с сайта проекта и заменить эмбеддинги в модели выше на эмбеддинги fastText.

Бонус: попробуйте взять модель с сайта проекта Rusvetores: https://rusvectores.org/ru/models/

In [95]:
!pip install fasttext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 KB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fasttext: filename=fasttext-0.6.0-cp38-cp38-linux_x86_64.whl size=1070768 sha256=ab20e53bc87f3fbbb88985568db03d6945ca282e092444184d6959ecb784d8bf
  Stored in directory: /root/.cache/pip/wheels/0c/75/aa/f731246516a4b4373cdcbe213982143b99afde374d269072d3
Successfully built fasttext


In [103]:

!pip install fastText

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [97]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!pip install fastText

Cloning into 'fastText'...
remote: Enumerating objects: 3930, done.
remote: Counting objects: 100% (943/943), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 3930 (delta 854), reused 806 (delta 806), pack-reused 2987
Receiving objects: 100% (3930/3930), 8.24 MiB | 30.47 MiB/s, done.
Resolving deltas: 100% (2505/2505), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [108]:
import fasttext
import fasttext.util

ModuleNotFoundError: ignored

In [ ]:
help(fasttext.util.download_model)

Help on function download_model in module fasttext.util.util:

download_model(lang_id, if_exists='strict', dimension=None)
    Download pre-trained common-crawl vectors from fastText's website
    https://fasttext.cc/docs/en/crawl-vectors.html



In [100]:
fasttext.util.download_model('ru', if_exists='ignore')
ft = fasttext.load_model('cc.ru.300.bin')

AttributeError: ignored

In [ ]:
ft['привет']

array([ 0.06434693, -0.01527086, -0.06963537, -0.03582602,  0.01471584,
       -0.03503159,  0.02701715,  0.04161827, -0.00033126,  0.00355259,
        0.06979205,  0.06205348,  0.05154078,  0.03831509, -0.02394784,
       -0.03954181, -0.00189653, -0.11174394, -0.0407712 ,  0.09289949,
       -0.07412342, -0.05209147,  0.02017231,  0.04837443,  0.02212641,
        0.00856511, -0.03055364,  0.04733564,  0.04380886,  0.03856769,
        0.03442968,  0.05576854,  0.01513439,  0.14055566,  0.03365337,
       -0.02920472, -0.10305687, -0.09332671,  0.03085899, -0.11067575,
       -0.08992791,  0.05850704, -0.017424  ,  0.00120653, -0.07153153,
        0.10312843, -0.08066262, -0.00642456,  0.04408539, -0.05728461,
       -0.0179531 ,  0.03936698,  0.04778077, -0.04907751, -0.00909553,
        0.05588715, -0.00236535,  0.04878682, -0.01769035,  0.03295048,
        0.00906604,  0.08772802,  0.02970458, -0.04903899, -0.03025401,
       -0.04151824,  0.04931813, -0.02804473,  0.05716789,  0.03

In [ ]:
x = 'привет всем слушателям курса'
get_tweet_embedding(x, model=ft, embedding_size=300)

array([ 2.84749218e-02,  1.14055865e-02, -1.54750008e-02,  6.10717852e-03,
       -5.42343501e-03,  2.83443742e-03,  2.40256451e-03,  1.29073053e-02,
        3.05031866e-02, -1.99234379e-02,  6.13203850e-02,  4.42768331e-02,
        2.71531800e-02, -1.02064133e-02,  9.22483567e-04,  2.50384058e-02,
       -1.25383004e-02, -4.89095808e-02, -3.07890818e-02,  1.01918663e-01,
       -2.85800546e-02, -1.05811988e-01, -1.28629373e-02,  2.95597422e-02,
        2.13206490e-03,  1.26906892e-02, -2.97227059e-02,  2.77029723e-02,
       -1.21254625e-02, -4.76178443e-02, -6.68591424e-03,  3.05985650e-02,
        3.59081652e-02,  1.02970391e-01,  3.62780495e-02, -5.56655712e-02,
       -1.11200343e-01, -1.16946280e-01,  4.69890856e-02, -5.79430675e-02,
       -4.56299540e-03, -2.32621958e-03, -2.30524363e-03,  1.96370891e-02,
       -1.68996924e-02,  4.77626729e-02, -7.71877861e-02,  2.95996453e-02,
        3.40769021e-02, -3.43663241e-02,  5.55797149e-02,  1.05126291e-02,
        9.77615127e-03,  

In [ ]:
train['ft_embedding'] = train['lemmas'].apply(lambda x: get_tweet_embedding(x, model=ft, embedding_size=300))
print('train done')

test['ft_embedding'] = test['lemmas'].apply(lambda x: get_tweet_embedding(x, model=ft, embedding_size=300))

train done


In [ ]:
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(list(train['ft_embedding'].values), y_train)

pred = clf.predict(list(test['ft_embedding'].values))
accuracy_score(pred, y_test)

0.8709408825978351

In [ ]:
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.95      0.87      0.91      2618
           1       0.71      0.88      0.79       985

    accuracy                           0.87      3603
   macro avg       0.83      0.87      0.85      3603
weighted avg       0.89      0.87      0.87      3603



### fastText как классификатор

fastText также можно использовать в режиме классификатора:

In [ ]:
train.sample()

,comment,toxic,lemmas,w2v_embedding,ft_embedding
5404,"да ну, кем считается то? у нас, швабов, пожест...",1.0,считаться шваб жёсткий лично думать,"[0.05510629341006279, 0.010897617496084422, 0....","[0.04708024859428406, 0.0034161420539021493, 0..."


In [ ]:
with open('train_ft.txt', 'w') as f:
    for label, lemmas in list(zip(
        train['toxic'], train['lemmas']
    )):
        f.write(f"__label__{int(label)} {lemmas}\n")
        #print(f"__label__{int(label)} {lemmas}")

with open('test_ft.txt', 'w') as f:
    for label, lemmas in list(zip(
        train['toxic'], train['lemmas']
    )):
        f.write(f"__label__{int(label)} {lemmas}\n")

In [ ]:
!tail train_ft.txt

__label__0 операция спирт тампон скальпель доктор выбрасывать пригодиться
__label__0 думать уместить знание мочь дать любой родитель домашний условие
__label__1 татарин народец гнилой весьма весьма
__label__0 красиво постановка театральный рыжий китайский очень доставить
__label__0 печально близкий нормальный любящий человек превращаться
__label__0 мама группа выпуск просяк случаться разный ребёнок ребёнок разовый акция время время третий частенько родитель прекрасно курс особенность каждый конкретный ребёнок
__label__1 сука тупой дегенарта видео съести свой старый куколд жухлый сморчок друг друг теребить
__label__1 племя украинец особенно западный детство прививаться мысль самый умный ловко наебал значит молодец понятие подлость честь отсутствовать нацело поэтому маленький дружок весь твой натужный изворотливость работать пообщаться пять минута любой россия понять сорт иметь дело услышать мягкий акцент твой речь
__label__0 пост жадность человек оплатить предоставить халява человек пре

In [ ]:
#help(fasttext.train_supervised)

In [ ]:
classifier = fasttext.train_supervised('train_ft.txt')#, 'model')
result = classifier.test('test_ft.txt')
print('P@1:', result[1])#.precision)
print('R@1:', result[2])#.recall)
print('Number of examples:', result[0])#.nexamples)

P@1: 0.9790914978258858
R@1: 0.9790914978258858
Number of examples: 10809


Read 0M words
Number of words:  26960
Number of labels: 2
Progress: 100.0% words/sec/thread: 1258741 lr:  0.000000 avg.loss:  0.295754 ETA:   0h 0m 0s


In [ ]:
pred = classifier.predict(list(test['lemmas']))[0]
pred = [int(label[0][-1]) for label in pred]

accuracy_score(list(y_test), pred)

0.8717735220649458